In [1]:
!pip install openai langchain faiss-cpu
!pip install langchain
!pip install openai
!pip install python-magic
!pip install chromadb
!pip install unstructured
!pip install tiktoken
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.6/934.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-w

In [2]:
import os
import nltk
import requests
import json
import openai

In [3]:
os.environ['OPENAI_API_KEY'] = '...'
openai.api_key = os.environ['OPENAI_API_KEY']

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
class Document:
    def __init__(self, text):
        self.page_content = text
        self.metadata = {}

In [5]:
def fetch_articles(query, rows=10):
    url = "https://api.crossref.org/works"
    params = {
        "query": query,
        "rows": rows
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    articles = data["message"]["items"]
    return articles

def generate_response(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1000,
        n=1,
        stop=None,
        temperature=0.7,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].text.strip()

In [7]:
# Get user input for the search term
search_term = input("Enter a term to search for related articles: ")

# Fetch articles related to the search term
articles = fetch_articles(search_term)

# Generate curation of articles using GPT-3
curation_prompt = f"Curate publications about {search_term} from the Crossref API and provide the correct citation, working link, and authors for each article in the response:\n\n"
curation_prompt += "\n".join(f"- {article['title'][0]}, {', '.join(author['given'] + ' ' + author['family'] for author in article.get('author', []))}, {article['DOI']}" for article in articles)

# Generate response using GPT-3
response = generate_response(curation_prompt)
print(response)

Enter a term to search for related articles: information systems
1. Information about computer-assisted information systems,  B.W. Lampson, 10.1016/0306-4379(75)90019-8, https://doi.org/10.1016/0306-4379(75)90019-8

2. Information about computer-assisted information systems, M.M. Reddy, 10.1016/0306-4379(75)90013-7, https://doi.org/10.1016/0306-4379(75)90013-7

3. Information about computer-assisted information systems, C.C. Gotlieb, 10.1016/0306-4379(76)90007-7, https://doi.org/10.1016/0306-4379(76)90007-7

4. Information about computer-assisted information systems, J.A.N. Lee, 10.1016/0306-4379(82)90010-2, https://doi.org/10.1016/0306-4379(82)90010-2

5. New submission procedure at information systems, S.A. Jones, 10.1016/s0306-4379(01)00028-x, https://doi.org/10.1016/s0306-4379(01)00028-x

6. Information systems theory, Börje Langefors, 10.1016/0306-4379(77)90009-6, https://doi.org/10.1016/0306-4379(77)90009-6

7. Distributed information systems, rep. 370, Zdzislaw Pawlak, 10.1016/0